In [5]:
import json
import keras
import numpy as np
def extractGenres(row):
    json_dict_list = json.loads(row.genres)
    genres = [d['name'].lower() for d in json_dict_list if 'name' in d]
    return " ".join(genres)

In [6]:
import pandas as pd
df = pd.read_csv('datasets/tmdb_5000_movies.csv')
X = [str(p.overview) for p in df.itertuples()]
y = [extractGenres(p).rstrip() for p in df.itertuples()]

In [7]:
from keras.preprocessing.text import Tokenizer
#from keras.sequences import pad_sequences
Xtokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True,)
Ytokenizer = Tokenizer(filters=' ',lower=True,)

Xtokenizer.fit_on_texts(X)
plot_matrix = Xtokenizer.texts_to_matrix(X)


Ytokenizer.fit_on_texts(y)
genre_matrix = Ytokenizer.texts_to_matrix(y)

input_size = plot_matrix.shape[1]
output_size = genre_matrix.shape[1]
print(input_size,output_size)

22811 23


In [12]:
split_ratio = 0.8
split_index = int(len(X)*split_ratio)
X_train,X_test = np.array(plot_matrix[:split_index]) , np.array(plot_matrix[split_index:])
y_train,y_test = np.array(genre_matrix[:split_index]) , np.array(genre_matrix[split_index:])

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adadelta
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=input_size))
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(output_size, activation='sigmoid'))
model.summary()

adadelta = Adadelta(lr=0.1, rho=0.95, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=adadelta,metrics=['top_k_categorical_accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               11679744  
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 23)                2967      
Total params: 11,748,375
Trainable params: 11,748,375
Non-trainable params: 0
_________________________________________________________________


In [14]:
from keras.callbacks import Callback,ModelCheckpoint,EarlyStopping
checkpoint=ModelCheckpoint(filepath=None,monitor='val_loss',mode='min',verbose=1)

In [15]:
import numpy as np
model.fit(X_train, y_train,epochs=50, batch_size=128, verbose=1,callbacks=[])

Epoch 1/50
3842/3842 [==============================] - 8s 2ms/step - loss: 8.6234 - top_k_categorical_accuracy: 0.5401
Epoch 2/50
3842/3842 [==============================] - 7s 2ms/step - loss: 8.2366 - top_k_categorical_accuracy: 0.8004
Epoch 3/50
3842/3842 [==============================] - 7s 2ms/step - loss: 7.6421 - top_k_categorical_accuracy: 0.9151
Epoch 4/50
3842/3842 [==============================] - 7s 2ms/step - loss: 7.4325 - top_k_categorical_accuracy: 0.9412
Epoch 5/50
3842/3842 [==============================] - 7s 2ms/step - loss: 7.3676 - top_k_categorical_accuracy: 0.9472
Epoch 6/50
3842/3842 [==============================] - 8s 2ms/step - loss: 7.3243 - top_k_categorical_accuracy: 0.9430
Epoch 7/50
3842/3842 [==============================] - 7s 2ms/step - loss: 7.3031 - top_k_categorical_accuracy: 0.9438
Epoch 8/50
3842/3842 [==============================] - 7s 2ms/step - loss: 7.2671 - top_k_categorical_accuracy: 0.9433
Epoch 9/50
3842/3842 [==================

In [16]:
from keras.models import load_model
score = model.evaluate(np.array(X_test), np.array(y_test), verbose=0)
model.save('el_model.h5')
print("Accuracy: ", score)

Accuracy:  [5.083805448430882, 0.809573361082206]


In [17]:
comments = ['An extraordinary target requires an exraordinary innings !! Bravo Kohli !!',
            'I love my dog',
            'my whole body feels itchy and like its on fire',
            'Robbery Attempted at my bank today !! Stole 2m US Dollars']
comments_test = Xtokenizer.texts_to_matrix(comments)
y_probs = model.predict(np.array(comments_test))

In [18]:
top_k = 3
probs_to_indices = lambda ypb,k:np.argpartition(ypb,-k)[-k:]

In [19]:
genre_to_index = Ytokenizer.word_index
index_to_genre = [0]*output_size
genre_to_index[''] = 0
for k,v in genre_to_index.items():
    index_to_genre[v] = k    
indices_to_genres = lambda ind:[index_to_genre[i] for i in ind]

In [20]:
output_genres = [indices_to_genres(probs_to_indices(p,top_k)) for p in y_probs]

In [21]:
for out in output_genres:
    print(out)

['drama', 'thriller', 'action']
['drama', 'comedy', 'romance']
['comedy', 'thriller', 'drama']
['crime', 'comedy', 'drama']
